#  Recommendation de film basée sur le contenu  (Content-Based)


In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")

## Le but est de construre  un moteur de recommandation basé sur le contenu qui calcule la similarité entre les films en fonction de leur genre. Il suggérera les films les plus similaires à un film particulier en fonction de son genre. Pour ce faire, on utilise le fichier movies.csv.

In [3]:
# Décomposer la chaîne de caractère "genre" en un tableau de chaînes de caractères
movies['genres'] = movies['genres'].str.split('|')
# Convertire genres en string value
movies['genres'] = movies['genres'].fillna("").astype('str')

## La fonction TfidfVectorizer de scikit-learn transforme le texte en vecteurs de caractéristiques pouvant être utilisés comme entrée de l'estimateur

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

InvalidParameterError: The 'min_df' parameter of TfidfVectorizer must be a float in the range [0.0, 1.0] or an int in the range [1, inf). Got 0 instead.

## On utilise ensuite **[Cosine Similarity] pour calculer pour calculer une quantité numérique qui indique la similarité entre deux films. Puisque nous avons utilisé le vecteur TF-IDF, le calcul du produit de points nous donnera directement le score de similarité cosinus. Par conséquent, nous utiliserons le **noyau_linéaire** de sklearn au lieu de cosinus_similarities car il est beaucoup plus rapide.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

## Remarque: Si vous rencontrez des problèmes de mémoire ou d'exécution prolongée, vous pouvez calculer uniquement les similarités pour un sous-ensemble d'éléments ou pour les k-plus proches voisins

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

# Exemple : calculer uniquement les similarités pour un échantillon
subset_indices = np.random.choice(tfidf_matrix.shape[0], size=100, replace=False)
subset_tfidf = tfidf_matrix[subset_indices]
cosine_sim_subset = linear_kernel(subset_tfidf, tfidf_matrix)


## On dispose à présent d'une matrice de similarité cosinus par paire pour tous les films du jeu de données. L'étape suivante consiste à écrire une fonction qui renvoie les 20 films les plus similaires sur la base du score de similarité cosinus.

In [ ]:

titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])


def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

## Essayons d'obtenir les meilleures recommandations pour quelques films et voyons si elles sont bonnes.

In [ ]:
genre_recommendations('Good Will Hunting (1997)').head(20)

In [ ]:
genre_recommendations('Toy Story (1995)').head(20)

In [ ]:
genre_recommendations('Saving Private Ryan (1998)').head(20)

On obtient une liste de recommandations pas trop mauvaise pour **Good Will Hunting** (Drame), **Toy Story** (Animation, Enfants, Comédie), et **Saving Private Ryan** (Action, Thriller, Guerre).

Dans l'ensemble, voici les avantages de la recommandation basée sur le contenu :
* Pas besoin de données sur les autres utilisateurs, donc pas de problèmes de démarrage à froid ou de rareté.
* Peut recommander aux utilisateurs ayant des goûts uniques.
* Possibilité de recommander des articles nouveaux et impopulaires.
* Possibilité de fournir des explications sur les éléments recommandés en énumérant les caractéristiques du contenu à l'origine de la recommandation (dans le cas présent, les genres de films).

Cette approche présente toutefois certains inconvénients :
* Il est difficile de trouver les caractéristiques appropriées.
* Elle ne permet pas de recommander des éléments en dehors du profil de contenu de l'utilisateur.
* Impossibilité d'exploiter les jugements de qualité des autres utilisateurs.


## Collaborative Filtering Recommendation Model
Dans ce qui suit et en complément du TP1, nous allons utiliser ici le filtrage collaboratif basé sur la mémoire pour faire des recommandations aux utilisateurs de films.
L'idée est de construire une matrice de similarité.  la **matrice de similarité utilisateur** se compose de certaines mesures de distance qui mesurent la similarité entre deux paires d'utilisateurs. De même, la **matrice de similarité des items** mesure la similarité entre deux paires d'items.

Trois mesures de similarité de distance sont généralement utilisées dans le cadre du filtrage collaboratif :
1. **Similarité Jaccard** :
    * La similarité est basée sur le nombre d'utilisateurs qui ont évalué les éléments A et B divisé par le nombre d'utilisateurs qui ont évalué soit A soit B
    * Elle est typiquement utilisée lorsque nous n'avons pas d'évaluation numérique mais juste une valeur booléenne, comme l'achat d'un produit ou le clic sur une annonce.

2. **la similarité cosinus** : (comme dans le système basé sur le contenu)
    * La similarité est le cosinus de l'angle entre les 2 vecteurs des vecteurs des articles A et B.
    * Plus les vecteurs sont proches, plus l'angle est petit et plus le cosinus est grand.

3. **Similarité de Pearson** :
    * La similarité est le coefficient de Pearson entre les deux vecteurs.

Nous allons tester la **similarité de Pearson** dans cette mise en œuvre.

### Mise en œuvre
On utilise d'abord le fichier **ratings.csv** car il contient l'identifiant de l'utilisateur, l'identifiant du film et les évaluations. Ces trois éléments sont tout ce dont on a besoin pour déterminer la similarité des utilisateurs sur la base de leurs évaluations d'un film particulier.

On commence par un  traitement rapide des données :

In [ ]:
# remplir les valeurs NaN avec des 0
ratings['userId'] = ratings['userId'].fillna(0)
ratings['movieId'] = ratings['movieId'].fillna(0)

# Remplacer les valeurs NaN dans la colonne de notation par la moyenne de toutes les valeurs
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())

En raison de la puissance de calcul limitée, nous n'utiliiserons qu'un échantillon aléatoire de 20 000 évaluations (2 %) sur les 1 million d'évaluations.

In [ ]:

small_data = ratings.sample(frac=0.02)

print(small_data.info())

On utilise  la bibliothèque **scikit-learn** pour diviser l'ensemble de données en test et train.  La bibliothèque **Cross_validation.train_test_split** mélange et divise les données en deux ensembles de données en fonction du pourcentage d'exemples de test, qui dans ce cas est de 0,2.

In [ ]:

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(small_data, test_size=0.2, random_state=42)

Nous devons créer une matrice utilisateur-item. Comme nous avons divisé les données en test et train, on doit créer deux matrices. La matrice de train contient 80 % des évaluations (ratings) et la matrice de test contient 20 % des évaluations.

In [ ]:
# Créer deux matrices utilisateur-item, l'une pour le training et l'autre pour le test.

# Convertir un DataFrame en tableau NumPy
train_data_matrix = train_data[['userId', 'movieId', 'rating']].to_numpy()
test_data_matrix = test_data[['userId', 'movieId', 'rating']].to_numpy()


print(train_data_matrix.shape)
print(test_data_matrix.shape)

La fonction **pairwise_distances** de sklearn permet de  calculer le [coefficient de corrélation de Pearson]. Cette méthode offre un moyen sûr de prendre une matrice de distance en entrée

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# Matrice de similarité des users
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

In [ ]:
# Matrice de similarité des items
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:4, :4])

Avec la matrice de similarité on peut maintenant prédire les évaluations qui n'ont pas été incluses dans les données. À l'aide de ces prédictions, on peut ensuite les comparer aux données de test pour tenter de valider la qualité de notre modèle de recommandation.

Dans le cas du user-user CF, on considère la similarité entre deux utilisateurs (A et B, par exemple) comme des poids qui sont multipliés par les évaluations d'un utilisateur B similaire (corrigées en fonction de l'évaluation moyenne de cet utilisateur). On doit également normaliser le tout pour que les notes restent comprises entre 1 et 5 et, dans une dernière étape, additionner les notes moyennes de l'utilisateur que nous essayons de prédire. L'idée est que certains utilisateurs peuvent avoir tendance à donner des notes élevées ou faibles à tous les films. La différence relative entre les notes attribuées par ces utilisateurs est plus importante que les valeurs absolues.


In [ ]:
# Fonction pour prédir des notes
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

### Evaluation
Il existe de nombreuses mesures d'évaluation, mais l'une des mesures les plus populaires utilisées pour évaluer la précision des évaluations prédites est l'**erreur quadratique moyenne de la racine (RMSE)**. Nous allons utiliser la fonction **mean_square_error (MSE)** de sklearn, où la RMSE est simplement la racine carrée de la MSE.

$$\mathit{RMSE} =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}$$.

On utilisera la fonction **moyenne des erreurs au carré** de scikit-learn comme mesure de validation. En comparant le filtrage collaboratif basé sur l'utilisateur et le filtrage collaboratif basé sur l'item.


In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Fonction de calcul de RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [ ]:
# Prédire les évaluations sur les données d'apprentissage avec les deux scores de similarité
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

# RMSE sur les données de test
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

In [ ]:
# RMSE sur le train
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))

La RMSE dest une mesure qui permet d'évaluer dans quelle mesure le signal et le bruit sont expliqués par le modèle. Nous obtenons un RMSE  assez élevé lié à un sur-apprentissage.

Dans l'ensemble, le filtrage collaboratif basé sur la mémoire est facile à mettre en œuvre et produit une qualité de prédiction raisonnable. Cependant, cette approche présente certains inconvénients :

* Elle n'aborde pas le problème bien connu du démarrage à froid, c'est-à-dire lorsqu'un nouvel utilisateur ou un nouvel élément entre dans le système. 
* Elle ne peut pas traiter les données éparses, ce qui signifie qu'il est difficile de trouver des utilisateurs qui ont évalué les mêmes articles.
* Il souffre de l'arrivée de nouveaux utilisateurs ou d'éléments qui n'ont pas été évalués.
* Il a tendance à recommander des articles populaires.